# Detecting NYC Traffic Anomalies Final Models Notebook

In this notebook I will walkthrough code to automate fitting the final model I created in the last notebook to all of the senors. If you are wondering how I got to the model we will be using, I recommend looking at the <b>Modeling Notebook</b> before continuing on with this one.

## Libraries Needed

In [3]:
#import required libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import folium
from keras.losses import mean_squared_error
import keras
import tensorflow as tf
import pickle
import sqlite3

## Establish Connection to Database

We want to connect to a database because there is important informationt that we will generate that we want to be able to keep track of. For example, we want to be able to store the metrics for the final model of each sensor so we can do further analysis on the overall state of our system. I'm using sqlite3 for my database, but you can use any one you want. However, the code will probably be slightly different.

In [4]:
# Establish a connection to the database you want to use. If you are creating a new database
# the code is the same. Just name the database in a way that makes sense to you and it will
# be created in the same folder as this notebook.
conn = sqlite3.connect('traffic_flow.db')

# Instantiate a cursor instance to interact with your database
c = conn.cursor()

In [14]:
# Create table - MODEL_METRICS
c.execute('''CREATE TABLE MODEL_METRICS
             ([sensor_id] INTEGER,
             [date] DATETIME,
             [data_from] DATETIME,
             [data_to] DATETIME,
             [training_mean] FLOAT,
             [training_std] FLOAT,
             [baseline_loss] FLOAT,
             [model_train_loss] FLOAT,
             [model_test_loss] FLOAT,
             PRIMARY KEY (sensor_id, date))''')
          

In [12]:
c.execute('''DROP TABLE MODEL_METRICS''')

In [15]:
# commit the changes
conn.commit()

## Import Data

In [8]:
with open('sensor_dfs.pickle', 'rb') as handle:
    sensor_dfs = pickle.load(handle)

In [9]:
# initialize the steps we will use for each sensor (explained in Modeling Notebook)
X_steps = 18
y_steps = 3

## Training a Model for Each Sensor

In [ ]:
def sensor_etl(df, ID, filename, X_steps, y_steps):
    """
    
    
    """
    #file suffix
    if start_year == end_year:
        suffix = str(start_year)
    else:
        suffix = str(start_year) + '_' + str(end_year)
        
    #import libraries
    from IPython.display import clear_output

    # Group the time series into weekly buckets
    sens_weekly = sens_clean['SPEED'].resample('7D')

    # Take the mean of each group 
    sens_weekly_mean = sens_weekly.mean()
    
    #plot the generic trend and save the figure
    fig1, ax1 = plt.subplots(figsize=(16,8))
    sens_weekly_mean.plot(color='blue', ax=ax1);
    fig1.set_facecolor('white')
    fig1.suptitle('Weekly Traffic Speeds (' + filename + ')')
    ax1.grid(axis='y')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Average Traffic Speed (MPH)')
    ax1.set_facecolor('white')
    for x in ['bottom', 'top', 'right', 'left']:
        ax1.spines[x].set_color('black')
    
    plt.savefig(filename + '_weekly_trend_' + suffix, bbox_inches='tight', facecolor='white')
    plt.close()

    #aggregate all of the data to each weekday and plot that data
    weekly_agg = day_of_week_agg(sens_clean)

    fig2, ax2 = plt.subplots(figsize=(16,8));
    for i in range(7):
        colors= ['blue', 'red', 'orange', 'green', 'gray', 'purple', 'darkblue']
        weekly_agg[i]['SPEED'].plot(ax=ax2, color=colors[i], alpha=0.75);
    color='black'   
    fig2.legend(labels=['Monday', 'Tuesday', 'Wednesday', 'Thurday', 'Friday', 'Saturday', 'Sunday']
               , edgecolor='black', framealpha=1, loc=(.85,.101), facecolor='white')
    fig2.set_facecolor('white')
    fig2.suptitle('Average Traffic Speeds Per Day (' + filename + ')', color=color)
    ax2.grid(axis='y')
    ax2.set_xticks([287, 251, 215, 179, 143, 107, 71, 35, 0])
    ax2.set_xticklabels(['12am', '9pm', '6pm', '3pm', '12pm', '9am', '6am', '3am', '12am'], color=color)
    ax2.set_xlabel('Time', color=color)
    ax2.set_ylabel('Average Traffic Speed (MPH)', color=color)
    ax2.set_facecolor('white')
    ax2.tick_params(axis='y', colors=color)
    for x in ['bottom', 'top', 'right', 'left']:
        ax2.spines[x].set_color('black')
    plt.savefig(filename + '_weekdays_' + suffix, bbox_inches='tight', facecolor='white')
    plt.close()
    

    #standardize data and split into train, val and test sets
    X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_dfs(dataframe = sens_clean, column = 'SPEED', X_steps = X_steps, y_steps = y_steps, standardize=True)

    
    #------MODELING------
    #Baseline
    y_pred = X_val[:, -y_steps:]

    baseline_loss = np.mean(mean_squared_error(y_val, y_pred))

    #Final Model
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[X_steps, 1]),
        keras.layers.Dense(40),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(40),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(20),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(y_steps)
    ])

    model.compile(optimizer='Adam', loss=tf.keras.losses.MeanSquaredError())

    history = model.fit(X_train, y_train, epochs = 5, verbose=0)
    train_loss = history.history['loss'][-1]
    val_loss = model.evaluate(X_val, y_val, verbose=0)
    clear_output(wait=True)
    
    
    c.execute(f'''
                    INSERT INTO MODEL_PERFORMANCE_2015_2020 (sensor_id,
                                                   model4_train_loss,
                                                   model4_test_loss,
                                                   model5_train_loss,
                                                   model5_test_loss,
                                                   final_model)
                                VALUES (
                                        {ID},
                                        {date},
                                        {data_from},
                                        {data_to},
                                        {training_mean},
                                        {training_std},
                                        {baseline_loss}
                                        {train_loss},
                                        {val_loss})

             ''')
    
    conn.commit()
    
    model.save(filename + '_model_' + suffix)
      
    clear_output(wait=True)